In [4]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import make_blobs
import numpy as np
import random 


#tf.enable_eager_execution()


def gen_cluster_data_list(Cv, Lv, Nv, Mv):
    Tr = []
    Ts = []
    C = Cv  # number of categories
    L = Lv   # number of centers
    N = Nv  # number of elements
    M = Mv  # number of dimensions
    X, y = make_blobs(n_samples=N, centers=L, n_features=M,cluster_std=.5, random_state=11)
    cmap = []
    for _ in range(L):
        cmap.append(random.randint(0,C-1))
    cols = []
    for i in range(N):
        cols.append(cmap[y[i]])

    for i in range(int(0.9*N)):
        row = [X[i,j] for j in range(M)]
        row.append(cols[i])
        Tr.append(row)
    
    for i in range(int(0.9*N)+1,N):
        row = [X[i,j] for j in range(M)]
        row.append(cols[i])
        Ts.append(row)
        
    return (Tr, Ts)

def normalize (train):
    mx = []
    mn = []
    for i in range(len(train[0])-1):
        mx.append(max([x[i] for x in train ]))
        mn.append(min([x[i] for x in train ]))
    for row in train:
        for i in range(len(row)-1):
            row[i] = (row[i] - mn[i]) / (mx[i] - mn[i]) 
    return train


def gen_data_array(Cv, Lv, Nv, Mv):
    C = Cv  # number of categories
    L = Lv   # number of centers
    N = Nv  # number of elements
    M = Mv  # number of dimensions
    (T1,T2)  = gen_cluster_data_list(C, L, N, M)

    T = normalize(T1)
    N = len(T)
    x2_train = np.zeros((N,M),dtype='float32')
    y2_train = np.zeros((N,C))
    for i in range(N):
        row = T[i]
        for j in range(M):
            x2_train[i,j] = row[j]
        y2_train[i,row[-1]] = 1

    Ts = normalize(T2)
    Ns = len(Ts)
    x2_test = np.zeros((Ns,M),dtype='float32')
    y2_test = np.zeros((Ns,C))
    for i in range(Ns):
        row = Ts[i]
        for j in range(M):
            x2_test[i,j] = row[j]
        y2_test[i, row[-1]] = 1
        
    return (x2_train,y2_train, x2_test, y2_test)

def gen_data_array_s(Cv, Lv, Nv, Mv):
    C = Cv  # number of categories
    L = Lv   # number of centers
    N = Nv  # number of elements
    M = Mv  # number of dimensions
    (T1,T2)  = gen_cluster_data_list(C, L, N, M)

    T = normalize(T1)
    N = len(T)
    x2_train = np.zeros((N,M),dtype='float32')
    y2_train = np.zeros((N,1))
    for i in range(N):
        row = T[i]
        for j in range(M):
            x2_train[i,j] = row[j]
        y2_train[i] = row[-1]

    Ts = normalize(T2)
    Ns = len(Ts)
    x2_test = np.zeros((Ns,M),dtype='float32')
    y2_test = np.zeros((Ns,1))
    for i in range(Ns):
        row = Ts[i]
        for j in range(M):
            x2_test[i,j] = row[j]
        y2_test[i] = row[-1]
        
    return (x2_train,y2_train, x2_test, y2_test)



In [15]:
# P 05
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow.keras.activations import sigmoid

class SSLayer(tf.keras.layers.Layer):
    def __init__(self,  num_outputs,activation=sigmoid):
        super(SSLayer, self).__init__()
        self.num_outputs = num_outputs
        self.activation = activation
        
    def build(self, input_shape):  
        self.batchsize = int(input_shape[0])
        self.kernel = self.add_weight("kernel",
                                      shape=[int(input_shape[-1]),
                                             self.num_outputs], 
                                      initializer=tf.keras.initializers.RandomNormal(stddev=.3))
        print ("kernel ", self.kernel)
        
        self.bias = self.add_weight("bias",
                                      shape=[self.num_outputs],
                                    initializer=tf.keras.initializers.RandomNormal(stddev=.5))
        
        print ("bias ", self.bias)
    

    def call(self, input):
        print (input)
        print (tf.ones([self.batchsize, self.num_outputs]))
        ddi = tf.multiply( tf.transpose(input), tf.ones([self.batchsize, self.num_outputs]) )
        ddd = self.kernel-ddi
        dd0 = tf.math.multiply(ddd, ddd)
        dd1 = tf.math.reduce_sum(dd0, axis =0)
        dd2 = tf.cast(dd1,tf.double)
        dd3 = tf.sqrt(dd2)
        dd3 = tf.cast(dd3,tf.float32)
        dd4 = tf.abs(self.bias)
        result = tf.math.divide_no_nan(dd3,dd4)    
        rr2 = tf.ones([self.batchsize, self.num_outputs]) -result
        rr3 = tf.math.scalar_mul(6,rr2)
        result = self.activation(rr3)
        return result


class NN_Model(Model):
    
  def __init__(self,c,l,n,m):
    self.C=c
    self.L=l
    self.N=n
    self.M=m
    super(NN_Model, self).__init__()
    self.d1 = SSLayer(4*self.M)
    self.d2 = Dense(self.C)

  def call(self, x):
    x = self.d1(x)
    return self.d2(x)

@tf.function
def train_step(datas, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(datas, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(datas, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(datas, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

C = 2
L = 10
N = 1000
M = 3

# Create an instance of the model
model = NN_Model(C,L,N,M)

loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
#loss_object = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
print ("--------------")
print (C, L, N, M)
(x_train,y_train,x_test,y_test) = gen_data_array(C, L, N, M)
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).batch(32)
print (train_ds)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

    
EPOCHS = 20

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for datas, labels in train_ds:
    print ("input:", datas)
    print ("labels", labels)
    train_step(datas, labels)

  for test_datas, test_labels in test_ds:
    test_step(test_datas, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )    


--------------
2 10 1000 3
<BatchDataset shapes: ((None, 3), (None, 2)), types: (tf.float32, tf.float64)>
input: tf.Tensor(
[[0.8914187  0.44162735 0.2808851 ]
 [0.8515105  0.46186763 0.24088585]
 [0.67635417 0.07768797 0.1056074 ]
 [0.942432   0.7850282  0.04864224]
 [0.13471572 0.22203955 0.6862668 ]
 [0.6970358  0.06602696 0.04721393]
 [0.06776198 0.5567025  0.9401386 ]
 [0.92003983 0.8369356  0.01939357]
 [0.8942079  0.41677353 0.26042283]
 [0.95389813 0.7809099  0.05973615]
 [0.24062414 0.03391621 0.46969435]
 [0.65640914 0.05278561 0.47191405]
 [0.7091805  0.         0.13205338]
 [0.86896616 0.43564287 0.25718904]
 [0.22771984 0.06400549 0.4711761 ]
 [0.93484247 0.93178266 0.16285257]
 [0.7452247  0.08793405 0.0679831 ]
 [0.8668542  0.41336656 0.2919149 ]
 [0.8878456  0.7762019  0.05588934]
 [0.74989474 0.42231864 0.4683498 ]
 [0.69134665 0.15128346 0.4693992 ]
 [0.35709557 0.226434   0.7199237 ]
 [0.33078822 0.22774236 0.7373164 ]
 [0.85176957 0.3695334  0.27421528]
 [0.35536852

ValueError: in user code:

    <ipython-input-9-a2d2b17f19c0>:65 train_step  *
        predictions = model(datas, training=True)
    <ipython-input-9-a2d2b17f19c0>:57 call  *
        x = self.d1(x)
    <ipython-input-9-a2d2b17f19c0>:30 call  *
        ddi = tf.multiply( tf.transpose(input), tf.ones([self.batchsize, self.num_outputs]) )
    C:\Users\kovacs\anaconda3\envs\proba\lib\site-packages\tensorflow\python\util\dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    C:\Users\kovacs\anaconda3\envs\proba\lib\site-packages\tensorflow\python\ops\math_ops.py:509 multiply
        return gen_math_ops.mul(x, y, name)
    C:\Users\kovacs\anaconda3\envs\proba\lib\site-packages\tensorflow\python\ops\gen_math_ops.py:6174 mul
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    C:\Users\kovacs\anaconda3\envs\proba\lib\site-packages\tensorflow\python\framework\op_def_library.py:742 _apply_op_helper
        op = g._create_op_internal(op_type_name, inputs, dtypes=Non

In [18]:
# P 04
from tensorflow.keras.activations import sigmoid

class SSLayer(tf.keras.layers.Layer):
    def __init__(self, num_inputs, num_outputs,activation=sigmoid):
        super(SSLayer, self).__init__()
        self.num_outputs = num_outputs
        self.kernel = self.add_weight("kernel",
                                      shape=[num_inputs,
                                             self.num_outputs], 
                                      initializer=tf.keras.initializers.RandomNormal(stddev=.3))
        
        self.bias = self.add_weight("bias",
                                      shape=[self.num_outputs],
                                    initializer=tf.keras.initializers.RandomNormal(stddev=.5))
        
        self.activation = activation

    def call(self, input):
        print ("kernel", self.kernel)
        print ("bias", self.bias)
        ddi = tf.multiply( tf.transpose(input), tf.ones([1, self.num_outputs]) )
        print ("ddi", ddi)
        ddd = self.kernel-ddi
        print ("ddd", ddd)
        dd0 = tf.math.multiply(ddd, ddd)
        print ("dd0", dd0)
        dd1 = tf.math.reduce_sum(dd0, axis =0)
        print ("dd1", dd1)
        dd2 = tf.cast(dd1,tf.double)
        print ("dd2", dd2)
        dd3 = tf.sqrt(dd2)
        dd3 = tf.cast(dd3,tf.float32)
        print ("dd3", dd3)
        dd4 = tf.abs(self.bias)
        print ("dd4", dd4)
        result = tf.math.divide_no_nan(dd3,dd4)    
        print ("result", result)
        rr2 = tf.ones([1, self.num_outputs]) -result
        print ("rr2", rr2)
        rr3 = tf.math.scalar_mul(6,rr2)
        print ("rr3", rr3)
        result = self.activation(rr3)
        print ("result", result)
        return result

M_input = 3
M_ouput = 5
#layer = SSLayer(M_input, M_ouput)
layer = SSLayer(M_input, M_ouput,activation=sigmoid)

print([var.name for var in layer.trainable_variables])

a = np.array([[0.2, 0.3, 0.4]],dtype="float32")
print ("out:")
print (layer(tf.constant(a)))



['kernel:0', 'bias:0']
out:
kernel <tf.Variable 'kernel:0' shape=(3, 5) dtype=float32, numpy=
array([[ 0.2523557 , -0.4805397 , -0.23506409,  0.3204259 ,  0.08248037],
       [-0.04309471, -0.03324677, -0.0968312 , -0.27222425,  0.2576189 ],
       [ 0.2890751 ,  0.2100241 , -0.10678596,  0.07982869, -0.51990956]],
      dtype=float32)>
bias <tf.Variable 'bias:0' shape=(5,) dtype=float32, numpy=
array([-0.15722479,  0.3852226 ,  0.24986114,  0.3118788 ,  0.06286097],
      dtype=float32)>
ddi tf.Tensor(
[[0.2 0.2 0.2 0.2 0.2]
 [0.3 0.3 0.3 0.3 0.3]
 [0.4 0.4 0.4 0.4 0.4]], shape=(3, 5), dtype=float32)
ddd tf.Tensor(
[[ 0.05235569 -0.6805397  -0.43506408  0.12042589 -0.11751963]
 [-0.3430947  -0.33324677 -0.3968312  -0.57222426 -0.04238111]
 [-0.1109249  -0.1899759  -0.506786   -0.32017133 -0.9199096 ]], shape=(3, 5), dtype=float32)
dd0 tf.Tensor(
[[0.00274112 0.46313432 0.18928075 0.0145024  0.01381086]
 [0.11771398 0.11105341 0.15747501 0.3274406  0.00179616]
 [0.01230433 0.03609084 0

In [66]:
#P 03


class SSLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(SSLayer, self).__init__()
        self.num_outputs = num_outputs

    def build(self, input_shape):
        self.kernel = self.add_weight("kernel",
                                      shape=[int(input_shape[-1]),
                                             self.num_outputs], 
                                      initializer=tf.keras.initializers.RandomNormal(stddev=4))
        
        self.bias = self.add_weight("bias",
                                      shape=[self.num_outputs])

    def call(self, input):
        return tf.matmul(input, self.kernel)

M_input = 3
M_ouput = 5
layer = SSLayer(M_ouput)

print([var.name for var in layer.trainable_variables])

a = np.array([[1.0, 2.3, 3.0]],dtype="float32")
print (layer(tf.constant(a)))

print([var.name for var in layer.trainable_variables])
print (layer.kernel, layer.bias)


[]
tf.Tensor([[ 10.911133    6.1613445 -17.000881   -8.46829   -11.545963 ]], shape=(1, 5), dtype=float32)
['ss_layer_13/kernel:0', 'ss_layer_13/bias:0']
<tf.Variable 'ss_layer_13/kernel:0' shape=(3, 5) dtype=float32, numpy=
array([[ 6.808793  , -2.8096771 , -0.8926972 ,  1.2995508 ,  4.6321363 ],
       [ 2.1581118 , -3.11415   ,  0.6261549 , -0.4685134 ,  0.15324706],
       [-0.28710544,  5.3778553 , -5.8494463 , -2.8967533 , -5.510189  ]],
      dtype=float32)> <tf.Variable 'ss_layer_13/bias:0' shape=(5,) dtype=float32, numpy=
array([ 0.15592939, -0.21368217, -0.21329248, -0.38312   , -0.35173374],
      dtype=float32)>


In [47]:
#P 02
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model

class NN_Model(Model):
    
  def __init__(self,c,l,n,m):
    self.C=c
    self.L=l
    self.N=n
    self.M=m
    super(NN_Model, self).__init__()
    self.d1 = Dense(4*self.M, input_shape=(M,), activation='relu')
    self.d2 = Dense(self.C)

  def call(self, x):
    x = self.d1(x)
    return self.d2(x)

@tf.function
def train_step(datas, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(datas, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(datas, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(datas, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

    
# Create an instance of the model
model = NN_Model(3,10,1000,3)

loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
#loss_object = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

(x_train,y_train,x_test,y_test) = gen_data_array(C, L, N, M)
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).batch(32)
print (train_ds)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

    
EPOCHS = 20

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for datas, labels in train_ds:
    train_step(datas, labels)

  for test_datas, test_labels in test_ds:
    test_step(test_datas, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )    


<DatasetV1Adapter shapes: ((?, 3), (?, 3)), types: (tf.float32, tf.float64)>
Epoch 1, Loss: 1.033753514289856, Accuracy: 49.77777862548828, Test Accuracy: 52.52525329589844
Epoch 2, Loss: 1.009682059288025, Accuracy: 47.0, Test Accuracy: 47.47474670410156
Epoch 3, Loss: 0.9931789040565491, Accuracy: 48.11111068725586, Test Accuracy: 46.46464538574219
Epoch 4, Loss: 0.9799463152885437, Accuracy: 46.22222137451172, Test Accuracy: 47.47474670410156
Epoch 5, Loss: 0.9679049253463745, Accuracy: 44.88888931274414, Test Accuracy: 54.54545593261719
Epoch 6, Loss: 0.955437958240509, Accuracy: 49.11111068725586, Test Accuracy: 54.54545593261719
Epoch 7, Loss: 0.9434725642204285, Accuracy: 49.5555534362793, Test Accuracy: 56.56565475463867
Epoch 8, Loss: 0.9311975240707397, Accuracy: 49.5555534362793, Test Accuracy: 57.57575607299805
Epoch 9, Loss: 0.921164870262146, Accuracy: 50.33333206176758, Test Accuracy: 59.5959587097168
Epoch 10, Loss: 0.9115208387374878, Accuracy: 52.999996185302734, Test

In [48]:
#P 01

import tensorflow as tf

C=3
L=10
N=1000
M=3
#(x_train,y_train,x_test,y_test) = gen_data_array_s(C, L, N, M)
(x_train,y_train,x_test,y_test) = gen_data_array(C, L, N, M)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(4*M, input_shape=(M,), activation='relu'),
  tf.keras.layers.Dense(C)
])

predictions = model(x_train[:1]).numpy()
print(x_train[:1],y_train[:1], predictions)

#loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20)
model.evaluate(x_test,  y_test, verbose=2)


[[0.8914187  0.44162735 0.2808851 ]] [[0. 0. 1.]] [[-0.48727185 -0.22519372  0.577533  ]]
Train on 900 samples
Epoch 1/20
900/900 [==============================] - 3s 3ms/sample - loss: 1.0977 - acc: 0.4044
Epoch 2/20
900/900 [==============================] - 0s 26us/sample - loss: 1.0759 - acc: 0.4044
Epoch 3/20
900/900 [==============================] - 0s 23us/sample - loss: 1.0630 - acc: 0.4044
Epoch 4/20
900/900 [==============================] - 0s 24us/sample - loss: 1.0532 - acc: 0.4044
Epoch 5/20
900/900 [==============================] - 0s 25us/sample - loss: 1.0433 - acc: 0.4544
Epoch 6/20
900/900 [==============================] - 0s 26us/sample - loss: 1.0337 - acc: 0.4800
Epoch 7/20
900/900 [==============================] - 0s 25us/sample - loss: 1.0257 - acc: 0.5044
Epoch 8/20
900/900 [==============================] - 0s 24us/sample - loss: 1.0186 - acc: 0.5044
Epoch 9/20
900/900 [==============================] - 0s 24us/sample - loss: 1.0115 - acc: 0.5044
Epoch 10

[0.9052198246271923, 0.57575756]

In [14]:
dd = [2,3,0,1,1,2]

oo = tf.one_hot(dd, 4)
print (oo)

tf.Tensor(
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]], shape=(6, 4), dtype=float32)


In [28]:
import math

mnist = tf.keras.datasets.mnist
x = tf.constant(3.0)
c1 = tf.constant(1.0)

with tf.GradientTape() as g:
    g.watch(x)
    #y = tf.math.sin(x * x)
    y = tf.math.divide(c1, c1 + tf.math.exp(-x))
dy_dx = g.gradient(y, x) 
print(dy_dx)


tf.Tensor(0.04517666, shape=(), dtype=float32)


In [8]:
a = np.array([[1, 2, 3], [1,4,1]])
ta = tf.constant(a)
b = np.array([[3, 2, 1],[1,3,2]])
tb = tf.constant(b)
tm = tf.math.multiply(ta-tb, ta-tb)
dd = tf.math.reduce_sum(tm, axis =1)
d2 = tf.cast(dd,tf.double)
d3 = tf.sqrt(d2)
dr = tf.fill([2], 2)
dr2= tf.cast(dr,tf.double)
print (dr)
out = tf.math.divide_no_nan(d3,dr2)
print (d3)
print (out)

tf.Tensor([2 2], shape=(2,), dtype=int32)
tf.Tensor([2.82842712 1.41421356], shape=(2,), dtype=float64)
tf.Tensor([1.41421356 0.70710678], shape=(2,), dtype=float64)


In [7]:
print (tf.multiply( tf.ones([4,1]),tf.constant([1.3,3.0,5.2])))

tf.Tensor(
[[1.3 3.  5.2]
 [1.3 3.  5.2]
 [1.3 3.  5.2]
 [1.3 3.  5.2]], shape=(4, 3), dtype=float32)


In [6]:
x = tf.constant([[1, 2]])
y = tf.constant([[2], [1]])
print (x)
print (y)
z = tf.matmul(y,x)
print (z)

tf.Tensor([[1 2]], shape=(1, 2), dtype=int32)
tf.Tensor(
[[2]
 [1]], shape=(2, 1), dtype=int32)
tf.Tensor(
[[2 4]
 [1 2]], shape=(2, 2), dtype=int32)


In [18]:
x = tf.ones([3,2,2])
y = tf.constant([  [3., 2.], [4.,3.]])
print (x)
print (y)
z = tf.matmul(x,y)
print (z)

tf.Tensor(
[[[1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]]], shape=(3, 2, 2), dtype=float32)
tf.Tensor(
[[3. 2.]
 [4. 3.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[[7. 5.]
  [7. 5.]]

 [[7. 5.]
  [7. 5.]]

 [[7. 5.]
  [7. 5.]]], shape=(3, 2, 2), dtype=float32)


In [38]:
In = tf.constant([[0.8914187,  0.44162735, 0.2808851 ],
 [0.8515105,  0.46186763, 0.24088585],
 [0.67635417, 0.07768797, 0.1056074 ],
 [0.942432,   0.7850282,  0.04864224],
 [0.13471572, 0.22203955, 0.6862668 ],
 [0.6970358,  0.06602696, 0.04721393],
 [0.06776198, 0.5567025,  0.9401386 ],
 [0.92003983, 0.8369356,  0.01939357],
 [0.8942079,  0.41677353, 0.26042283],
 [0.95389813, 0.7809099,  0.05973615]])
InD = tf.reshape(In,[10,1,3]) 
print (InD)
W = tf.constant([[ 0.64815485, -0.3559605 ,  0.20014353, -0.28112715,  0.09535905,
        -0.35034025,  0.12817992, -0.5305331 ,  0.03162672, -0.08267911,
        -0.02133827,  0.07015286],
       [-0.03005282, -0.5115979 ,  0.08203132,  0.16849105,  0.4781897 ,
         0.18680447, -0.02384542, -0.03511131,  0.14952978,  0.80725867,
         0.11003426, -0.39055902],
       [ 0.32035694, -0.15671906,  0.5563809 , -0.5291747 ,  0.15807222,
        -0.5098055 ,  0.26924753,  0.06294397, -0.16664729, -0.16909388,
         0.34415245,  0.04050118]])
WD = tf.stack([W] * 10)
print (WD)
R = tf.matmul(InD, WD)
print (R)

tf.Tensor(
[[[0.8914187  0.44162735 0.2808851 ]]

 [[0.8515105  0.46186763 0.24088585]]

 [[0.67635417 0.07768797 0.1056074 ]]

 [[0.942432   0.7850282  0.04864224]]

 [[0.13471572 0.22203955 0.6862668 ]]

 [[0.6970358  0.06602696 0.04721393]]

 [[0.06776198 0.5567025  0.9401386 ]]

 [[0.92003983 0.8369356  0.01939357]]

 [[0.8942079  0.41677353 0.26042283]]

 [[0.95389813 0.7809099  0.05973615]]], shape=(10, 1, 3), dtype=float32)
tf.Tensor(
[[[ 0.64815485 -0.3559605   0.20014353 -0.28112715  0.09535905
   -0.35034025  0.12817992 -0.5305331   0.03162672 -0.08267911
   -0.02133827  0.07015286]
  [-0.03005282 -0.5115979   0.08203132  0.16849105  0.4781897
    0.18680447 -0.02384542 -0.03511131  0.14952978  0.80725867
    0.11003426 -0.39055902]
  [ 0.32035694 -0.15671906  0.5563809  -0.5291747   0.15807222
   -0.5098055   0.26924753  0.06294397 -0.16664729 -0.16909388
    0.34415245  0.04050118]]

 [[ 0.64815485 -0.3559605   0.20014353 -0.28112715  0.09535905
   -0.35034025  0.12817992 -

In [31]:
vec = tf.constant([[1,2,3],[4,5,6]])  # shape=(2,3)
matrix = tf.stack([a] * 4)  # shape=(4,2,3)


print(vec)
print(matrix)

tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[[1 2 3]
  [1 4 1]]

 [[1 2 3]
  [1 4 1]]

 [[1 2 3]
  [1 4 1]]

 [[1 2 3]
  [1 4 1]]], shape=(4, 2, 3), dtype=int32)
